# Computer Vision

In [23]:
import gradio as gr
import requests
from PIL import Image, ImageDraw, ImageFont
import random
import platform

FEATURES = ['read', 'tags', 'smartCrops', 'people', 'caption', 'denseCaptions', 'objects' ]

def request_vision(features, image_path, language, smart_crops=""):
    endpoint = f"{ENDPOINT}computervision/imageanalysis:analyze"

    query_params = {
        "api-version" : "2024-02-01", 
        "features" : ','.join(features), 
        "language" : language
    }

    if "smartCrops" in features:
        query_params.update({
            "smartcrops-aspect-ratios" : smart_crops
        })
    # method : POST

    headers = {

        "Ocp-Apim-Subscription-Key" : comVi_key,
        "Content-Type" : "application/octet-stream"

    }

    # image_path로 이미지를 바이너리 형태로 읽어서 전송
    with open(image_path, "rb") as image_file:
        image_data = image_file.read()

    response = requests.post(endpoint, params=query_params, headers=headers, data=image_data)

    if response.status_code == 200:
        return response.json()

    else:
        return {}
    
def random_color():
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

def get_font():
    font_size=50
    if platform.system() == "Darwin":
        font = ImageFont.truetype("AppleGothic.ttf", size=font_size)
    elif platform.system() == "Windows":
        font = ImageFont.trutype("malgun.ttf", size=font_size)
    else:
        font = ImageFont.load_default(size=font_size)
    
    return font




def draw_image(image_path, features, response_json):
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    font = get_font()
    
    for feature in features:
        result_object = response_json["{}Result".format(feature)]
        color = random_color()

        if "values" in result_object.keys():
            block_list = result_object['values']
            for block in block_list:
                bounding_box = block.get('boundingBox', None)
                confidence = block.get('confidence', 1)

                if bounding_box and confidence > 0.80:
                    x, y, w, h = bounding_box['x'], bounding_box['y'], bounding_box['w'], bounding_box['h']
                    draw.rectangle([(x, y), (x+w, y+h)], outline=color, width=5)
                    draw.text((x+10, y+10), text=feature, fill = color, font=font)

    return image


# FEATURES = "caption"
# request_vision(FEATURES, "../ComputerVision/lotus.jpg")

with gr.Blocks() as demo:

    def change_image(image_path, features, language, smart_crops):

        if image_path:
            response_json = request_vision(features, image_path, language, smart_crops)
            image = draw_image(image_path, features, response_json)
            return image, response_json
        else:
            return None, None
        
    def change_features(features):
        print(features)
        # return features
        if "smartCrops" in features:
            return features, gr.update(visible=True)
        else:
            return features, gr.update(visible=False)



    def change_language(language, features):
        print(language)
        # return language
        # feature 받아서 value로 세팅
        print(set(features), features)
        value = list(set(FEATURES[:4]) & set(features))
        if "ko" == language:
            return language, gr.update(choices=FEATURES[:4:], value=value)

        else:
            return language, gr.update(choices=FEATURES, value=value)


    # def upload_image(image_path):
    #      print("upload image", image_path)

    # 언어 선택, 
    with gr.Column():
        language_radio = gr.Radio(label="언어 선택", choices=["en", "ko"], value="en")
        features_checkbox = gr.CheckboxGroup(label="Features", choices=FEATURES)
        smart_crop_textbox = gr.Textbox(label="Smart Crops", visible=False)

    with gr.Column():
        input_image = gr.Image(label="입력 이미지", type="filepath", height = 400)
        send_button = gr.Button("전송")


    
    with gr.Row():
        output_image = gr.Image(label="출력 이미지", type="pil", interactive=False)
        output_json = gr.JSON(label="결과 JSON")

        
    input_image.change(change_image, inputs=[input_image, features_checkbox, language_radio, smart_crop_textbox], outputs=[output_image, output_json])
    send_button.click(change_image, inputs=[input_image, features_checkbox, language_radio, smart_crop_textbox], outputs=[output_image, output_json])
    language_radio.change(change_language, inputs=[language_radio, features_checkbox], outputs=[language_radio, features_checkbox])
    features_checkbox.change(change_features, inputs=[features_checkbox], outputs=[features_checkbox, smart_crop_textbox])


demo.launch()
        





* Running on local URL:  http://127.0.0.1:7876

To create a public link, set `share=True` in `launch()`.


ko
set() []
en
set() []
['smartCrops']
['smartCrops', 'people']
['smartCrops', 'people', 'caption']
['smartCrops', 'people', 'caption', 'denseCaptions']
['smartCrops', 'people', 'caption', 'denseCaptions', 'objects']
ko
{'people', 'objects', 'denseCaptions', 'smartCrops', 'caption'} ['smartCrops', 'people', 'caption', 'denseCaptions', 'objects']
['smartCrops', 'people']
['smartCrops', 'people', 'tags']
['people', 'tags']
['people', 'tags', 'smartCrops']
